# Import data_utils

In [2]:
from climsim_utils.data_utils import *

# Instantiate class

In [3]:
# REPRODUCTION STEP: Needed to add path from current working directory to the parent directory of the ClimSim package
path_to_climslim = "../../Project3-Resources/"

In [4]:
grid_path = path_to_climslim + 'ClimSim/grid_info/ClimSim_low-res_grid-info.nc'
norm_path = path_to_climslim + 'ClimSim/preprocessing/normalizations/'

grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + 'inputs/input_mean.nc')
input_max = xr.open_dataset(norm_path + 'inputs/input_max.nc')
input_min = xr.open_dataset(norm_path + 'inputs/input_min.nc')
output_scale = xr.open_dataset(norm_path + 'outputs/output_scale.nc')

data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale)

# set variables to V1 subset
data.set_to_v1_vars()

# Load training and validation data

**Challenge II (Oct 22, 2023 - Oct 30, 2023)** Machine Learning exploration challenge.

### Data loader: The quickstart demo data set was proepared using the preprocessing/create_npy_data_splits.ipynb notebook. This notebook reads in data files from a local folder. 

Completed by Nashita Rahman

1. Could this be revised to read data files from the Huggingface hub? 
2. Could a user select data by specifying time range and subsampling scheme?

### 1. READING DATAFILES FROM THE HUGGINGFACE HUB

Steps:
1. Install the python library [huggingface_hub](https://huggingface.co/docs/huggingface_hub/index)
2. Import necessary packages for data loading
3. Use ``np.load``, hf_hub_download, the repository name (LEAP/subsampled_low_res), the repository type (dataset to account for LFS), and the specific file name to load the files in as needed

Changes to starter code:

1. New load statements (step 3)

```
scoring_input = np.load(hf_hub_download(repo_id=repo_name, filename="scoring_input.npy", repo_type=repo_type))
scoring_target = np.load(hf_hub_download(repo_id=repo_name, filename="scoring_target.npy", repo_type=repo_type))
train_input = np.load(hf_hub_download(repo_id=repo_name, filename="train_input.npy", repo_type=repo_type))
train_target = np.load(hf_hub_download(repo_id=repo_name, filename="train_target.npy", repo_type=repo_type))
val_input = np.load(hf_hub_download(repo_id=repo_name, filename="val_input.npy", repo_type=repo_type))
val_target = np.load(hf_hub_download(repo_id=repo_name, filename="val_target.npy", repo_type=repo_type))
```

2. Loading in training and validation data

```
# CHANGE: COMMENTED OUT
# data_path = '../../Project3-Resources/subsampled_low_res/'

# train_input_path = data_path + 'train_input.npy'
# train_target_path = data_path + 'train_target.npy'
# val_input_path = data_path + 'val_input.npy'
# val_target_path = data_path + 'val_target.npy'

# data.input_train = data.load_npy_file(train_input_path)
# data.target_train = data.load_npy_file(train_target_path)
# data.input_val = data.load_npy_file(val_input_path)
# data.target_val = data.load_npy_file(val_target_path)

data.input_train = train_input
data.target_train = train_target
data.input_val = val_input
data.target_val = val_target
```

3. Loading in scoring data

```
# CHANGE: COMMENTED OUT
# # REPRODUCTION STEP: Needed to add path from current working directory to the parent directory with the scoring input and target files
# data_path = '../../Project3-Resources/subsampled_low_res/'
# scoring_input_path = data_path + "scoring_input.npy"
# scoring_target_path = data_path + "scoring_target.npy"

# # path to target input
# data.input_scoring = np.load(scoring_input_path)

# # path to target output
# data.target_scoring = np.load(scoring_target_path)

data.input_scoring = scoring_input
data.target_scoring = scoring_target
```

In [5]:
# CHANGE: UNCOMMENT IF NOT INSTALLED
# pip install huggingface_hub

In [6]:
# Necessary imports
import numpy as np
from huggingface_hub import hf_hub_download

In [7]:
# Define the repository name and type
repo_name = "LEAP/subsampled_low_res"
repo_type = "dataset"  # This specifies that you are downloading from a dataset repository

# Download files and load them
scoring_input = np.load(hf_hub_download(repo_id=repo_name, filename="scoring_input.npy", repo_type=repo_type))
scoring_target = np.load(hf_hub_download(repo_id=repo_name, filename="scoring_target.npy", repo_type=repo_type))
train_input = np.load(hf_hub_download(repo_id=repo_name, filename="train_input.npy", repo_type=repo_type))
train_target = np.load(hf_hub_download(repo_id=repo_name, filename="train_target.npy", repo_type=repo_type))
val_input = np.load(hf_hub_download(repo_id=repo_name, filename="val_input.npy", repo_type=repo_type))
val_target = np.load(hf_hub_download(repo_id=repo_name, filename="val_target.npy", repo_type=repo_type))

In [8]:
# CHANGE: COMMENTED OUT
# data_path = '../../Project3-Resources/subsampled_low_res/'

# train_input_path = data_path + 'train_input.npy'
# train_target_path = data_path + 'train_target.npy'
# val_input_path = data_path + 'val_input.npy'
# val_target_path = data_path + 'val_target.npy'

# data.input_train = data.load_npy_file(train_input_path)
# data.target_train = data.load_npy_file(train_target_path)
# data.input_val = data.load_npy_file(val_input_path)
# data.target_val = data.load_npy_file(val_target_path)

data.input_train = train_input
data.target_train = train_target
data.input_val = val_input
data.target_val = val_target

# Train models

### Train constant prediction model

$\hat{y} = E[y_{train}]$

In [9]:
const_model = data.target_train.mean(axis = 0)

### Train multiple linear regression model

$\beta = {(X_{train}^TX_{train})}^{-1}X_{train}^Ty_{train}$

$\hat{y} = X_{input}^T \beta$

where $X_{train}$ and $X_{input}$ correspond to the training data and the input data you would like to inference on, respectively. $X_{train}$ and $X_{input}$ both have a column of ones concatenated to the feature space for the bias.


##### adding bias unit

In [10]:
X = data.input_train
bias_vector = np.ones((X.shape[0], 1))
X = np.concatenate((X, bias_vector), axis=1)

##### create model

In [ ]:
mlr_weights = np.linalg.inv(X.transpose()@X)@X.transpose()@data.target_train

### Train your models here

In [ ]:
### 
# train your model here
###

# Evaluate on validation data

### Set pressure grid

In [ ]:
data.set_pressure_grid(data_split = 'val')

### Load predictions

In [ ]:
# Constant Prediction
const_pred_val = np.repeat(const_model[np.newaxis, :], data.target_val.shape[0], axis = 0)
print(const_pred_val.shape)

# Multiple Linear Regression
X_val = data.input_val
bias_vector_val = np.ones((X_val.shape[0], 1))
X_val = np.concatenate((X_val, bias_vector_val), axis=1)
mlr_pred_val = X_val@mlr_weights
print(mlr_pred_val.shape)

# Load your prediction here

# Load predictions into data_utils object
data.model_names = ['const', 'mlr'] # add names of your models here
preds = [const_pred_val, mlr_pred_val] # add your custom predictions here
data.preds_val = dict(zip(data.model_names, preds))

### Weight predictions and target

1. Undo output scaling

2.  Weight vertical levels by dp/g

3. Weight horizontal area of each grid cell by a[x]/mean(a[x])

4. Convert units to a common energy unit

In [ ]:
data.reweight_target(data_split = 'val')
data.reweight_preds(data_split = 'val')

### Set and calculate metrics

In [ ]:
data.metrics_names = ['MAE', 'RMSE', 'R2', 'bias']
data.create_metrics_df(data_split = 'val')

### Create plots

In [ ]:
# set plotting settings
%config InlineBackend.figure_format = 'retina'
letters = string.ascii_lowercase

# create custom dictionary for plotting
dict_var = data.metrics_var_val
plot_df_byvar = {}
for metric in data.metrics_names:
    plot_df_byvar[metric] = pd.DataFrame([dict_var[model][metric] for model in data.model_names],
                                               index=data.model_names)
    plot_df_byvar[metric] = plot_df_byvar[metric].rename(columns = data.var_short_names).transpose()

# plot figure
fig, axes = plt.subplots(nrows  = len(data.metrics_names), sharex = True)
for i in range(len(data.metrics_names)):
    plot_df_byvar[data.metrics_names[i]].plot.bar(
        legend = False,
        ax = axes[i])
    if data.metrics_names[i] != 'R2':
        axes[i].set_ylabel('$W/m^2$')
    else:
        axes[i].set_ylim(0,1)

    axes[i].set_title(f'({letters[i]}) {data.metrics_names[i]}')
axes[i].set_xlabel('Output variable')
axes[i].set_xticklabels(plot_df_byvar[data.metrics_names[i]].index, \
    rotation=0, ha='center')

axes[0].legend(columnspacing = .9, 
               labelspacing = .3,
               handleheight = .07,
               handlelength = 1.5,
               handletextpad = .2,
               borderpad = .2,
               ncol = 3,
               loc = 'upper right')
fig.set_size_inches(7,8)
fig.tight_layout()

If you trained models with different hyperparameters, use the ones that performed the best on validation data for evaluation on scoring data.

## Evaluate on scoring data

#### Do this at the VERY END (when you have finished tuned the hyperparameters for your  model and are seeking a final evaluation)

### Load scoring data

In [ ]:
# CHANGE: COMMENTED OUT
# # REPRODUCTION STEP: Needed to add path from current working directory to the parent directory with the scoring input and target files
# data_path = '../../Project3-Resources/subsampled_low_res/'
# scoring_input_path = data_path + "scoring_input.npy"
# scoring_target_path = data_path + "scoring_target.npy"

# # path to target input
# data.input_scoring = np.load(scoring_input_path)

# # path to target output
# data.target_scoring = np.load(scoring_target_path)

data.input_scoring = scoring_input
data.target_scoring = scoring_target

### Set pressure grid

In [ ]:
data.set_pressure_grid(data_split = 'scoring')

### Load predictions

In [ ]:
# constant prediction
const_pred_scoring = np.repeat(const_model[np.newaxis, :], data.target_scoring.shape[0], axis = 0)
print(const_pred_scoring.shape)

# multiple linear regression
X_scoring = data.input_scoring
bias_vector_scoring = np.ones((X_scoring.shape[0], 1))
X_scoring = np.concatenate((X_scoring, bias_vector_scoring), axis=1)
mlr_pred_scoring = X_scoring@mlr_weights
print(mlr_pred_scoring.shape)

# Your model prediction here

# Load predictions into object
data.model_names = ['const', 'mlr'] # model name here
preds = [const_pred_scoring, mlr_pred_scoring] # add prediction here
data.preds_scoring = dict(zip(data.model_names, preds))

### Weight predictions and target

1. Undo output scaling

2.  Weight vertical levels by dp/g

3. Weight horizontal area of each grid cell by a[x]/mean(a[x])

4. Convert units to a common energy unit

In [ ]:
# weight predictions and target
data.reweight_target(data_split = 'scoring')

In [ ]:
# weight predictions and target
data.reweight_preds(data_split = 'scoring')

In [ ]:
# set and calculate metrics
data.metrics_names = ['MAE', 'RMSE', 'R2', 'bias']
data.create_metrics_df(data_split = 'scoring')

### Create plots

In [ ]:
# set plotting settings
%config InlineBackend.figure_format = 'retina'
letters = string.ascii_lowercase

# create custom dictionary for plotting
dict_var = data.metrics_var_scoring
plot_df_byvar = {}
for metric in data.metrics_names:
    plot_df_byvar[metric] = pd.DataFrame([dict_var[model][metric] for model in data.model_names],
                                               index=data.model_names)
    plot_df_byvar[metric] = plot_df_byvar[metric].rename(columns = data.var_short_names).transpose()

# plot figure
fig, axes = plt.subplots(nrows  = len(data.metrics_names), sharex = True)
for i in range(len(data.metrics_names)):
    plot_df_byvar[data.metrics_names[i]].plot.bar(
        legend = False,
        ax = axes[i])
    if data.metrics_names[i] != 'R2':
        axes[i].set_ylabel('$W/m^2$')
    else:
        axes[i].set_ylim(0,1)

    axes[i].set_title(f'({letters[i]}) {data.metrics_names[i]}')
axes[i].set_xlabel('Output variable')
axes[i].set_xticklabels(plot_df_byvar[data.metrics_names[i]].index, \
    rotation=0, ha='center')

axes[0].legend(columnspacing = .9, 
               labelspacing = .3,
               handleheight = .07,
               handlelength = 1.5,
               handletextpad = .2,
               borderpad = .2,
               ncol = 3,
               loc = 'upper right')
fig.set_size_inches(7,8)
fig.tight_layout()